
# 🧠 RAG sobre PDFs locales (sin scraping)

Este notebook **lee PDFs de una carpeta** (recursivo), los **trocea**, crea **embeddings**, los guarda en **Chroma**, y permite preguntar con una función `ask()`.

> Requisitos recomendados (mismo entorno):  
> `langchain>=0.2.5`, `langchain-community>=0.2.0`, `langchain-text-splitters>=0.2.0`, `langchain-openai>=0.1.0`  
> `chromadb>=0.5.0`, `tiktoken>=0.7.0`, `pypdf>=4`, `python-dotenv>=1.0.1`


In [1]:

# (Opcional) Instala dependencias
%pip install -U langchain langchain-community langchain-openai langchain-text-splitters chromadb tiktoken pypdf python-dotenv pyarrow fastparquet


  You can safely remove it manually.
  You can safely remove it manually.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



  Using cached chromadb-1.3.4-cp39-abi3-win_amd64.whl.metadata (7.4 kB)
  Using cached tiktoken-0.12.0-cp312-cp312-win_amd64.whl.metadata (6.9 kB)
  Using cached fastparquet-2024.11.0-cp312-cp312-win_amd64.whl.metadata (4.3 kB)
  Using cached build-1.3.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached uvicorn-0.38.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached posthog-5.4.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached onnxruntime-1.23.2-cp312-cp312-win_amd64.whl.metadata (5.3 kB)
  Using cached opentelemetry_api-1.38.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.38.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_sdk-1.38.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached bcrypt-5.0.0-cp39-abi3-win_amd64.whl.metadata (10 kB)
  Using cached kubernetes-34.1.0-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached mmh3-5.2.0-cp312-cp312-w

In [2]:

import os
from pathlib import Path
from typing import List

from dotenv import load_dotenv
load_dotenv()  # Carga variables de entorno (OPENAI_API_KEY, etc.)

# LangChain loaders, splitters, vectorstore, LLM/embeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

# Embeddings y modelo de chat (por defecto OpenAI)
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

# Utilidad
from tqdm import tqdm

print("✅ Entorno listo")


✅ Entorno listo


In [3]:

# 🛠️ Configuración
PDF_DIR = Path("./docs")             # <- Cambia a tu carpeta con PDFs
PERSIST_DIR = Path("./chroma_pdfs")  # Carpeta donde se guardará Chroma
PERSIST_DIR.mkdir(parents=True, exist_ok=True)

CHUNK_SIZE = 800
CHUNK_OVERLAP = 120
TOP_K = 4

# Modelos (ajusta si quieres otros)
EMBEDDING_MODEL = "text-embedding-3-large"
CHAT_MODEL = "gpt-4.1-mini"

# Verificar clave
assert os.getenv("OPENAI_API_KEY"), "Falta OPENAI_API_KEY en variables de entorno o .env"
print(f"📁 Carpeta PDFs: {PDF_DIR.resolve()}")
print(f"🗂️ Persistencia Chroma: {PERSIST_DIR.resolve()}")


📁 Carpeta PDFs: C:\Trainings\GenIA_trainings\agentic_rag_openai\docs
🗂️ Persistencia Chroma: C:\Trainings\GenIA_trainings\agentic_rag_openai\chroma_pdfs


In [4]:

def load_pdfs_from_dir(directory: Path, recursive: bool = True):
    pattern = "**/*.pdf" if recursive else "*.pdf"
    pdf_paths = sorted([p for p in directory.glob(pattern) if p.is_file()])
    all_docs = []
    for pdf in tqdm(pdf_paths, desc="Cargando PDFs"):
        try:
            docs = PyPDFLoader(str(pdf)).load()
            # Añadimos metadatos mínimos útiles
            for d in docs:
                d.metadata = d.metadata or {}
                d.metadata["source"] = str(pdf.resolve())
            all_docs.extend(docs)
        except Exception as e:
            print(f"⚠️ Error leyendo {pdf}: {e}")
    print(f"📚 Documentos (páginas) cargados: {len(all_docs)}")
    return all_docs

raw_docs = load_pdfs_from_dir(PDF_DIR, recursive=True)


Cargando PDFs: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:02<00:00,  2.39s/it]

📚 Documentos (páginas) cargados: 98


In [5]:

splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    separators=["\n\n", "\n", " ", ""],
)
chunks = splitter.split_documents(raw_docs)
print(f"🧩 Chunks generados: {len(chunks)}")
chunks[:2]  # vista rápida


🧩 Chunks generados: 217


[Document(metadata={'producer': 'PyPDF', 'creator': 'Google', 'creationdate': '', 'title': 'Generative AI Deployment and Monitoring.pptx', 'source': 'C:\\Trainings\\GenIA_trainings\\agentic_rag_openai\\docs\\generative-ai-deployment-and-monitoring.pdf', 'total_pages': 98, 'page': 0, 'page_label': '1'}, page_content='Lava 600 - Primary\n#FF3621\nRGB (255 ,54, 33)\nC0, M91, Y93, K0\nNavy 800 - Primary\n#1B3139\nRGB (27, 49, 57)\nC86, M65, Y57, K56\nMaroon 600\n#98102A\nRGB (152, 16, 42)\nC26, M100, Y84, K24\nYellow 600\n#FFAB00\nRGB (255, 171, 0)\nC0, M38, Y100, K0\nGreen 600\n#00A972\nRGB (0, 169, 114)\nC81, M6, Y74, K0\nBlue 600\n#2272B4\nRGB (34, 114,1 80)\nC86, M52, Y4, K0\nGray - Navigation\n#303F47\nRGB (48, 63, 71)\nC79, M62, Y54, K44\nGray – Text\n#5A6F77\nRGB (90, 111, 119)\nC68, M47, Y44, K14\nGray – Lines\n#DCE0E2\nRGB (220, 224, 226)\nC12, M7, Y8, K0\nPrimary palette\nSecondary palette\nOat Medium\n#EEEDE9\nRGB (238, 237, 233)\nC6, M4, Y6, K0\nOat Light\n#F9F7F4\nRGB (249, ,2

In [6]:

embeddings = OpenAIEmbeddings(model=EMBEDDING_MODEL)
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=str(PERSIST_DIR),
)
vectordb.persist()
print("💾 Chroma persistido")


💾 Chroma persistido


C:\Users\xavi.marti_bluetab\AppData\Local\Temp\ipykernel_33060\3314839376.py:7: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [11]:

from langchain_core.messages import SystemMessage, HumanMessage
retriever = vectordb.as_retriever(search_kwargs={"k": TOP_K})
llm = ChatOpenAI(model=CHAT_MODEL)

SYSTEM_PROMPT = (
    "Eres un asistente experto. Responde basándote EXCLUSIVAMENTE en el contexto entregado. "
    "Si la respuesta no está en las fuentes, di con claridad que no está disponible."
)

def _retrieve(query: str):
    """Función compatible: usa .invoke() y, si no existe, cae a .get_relevant_documents()."""
    if hasattr(retriever, "invoke"):
        return retriever.invoke(query)
    return retriever.get_relevant_documents(query)

def ask(query: str):
    # 1) Recuperación
    docs = _retrieve(query)

    # 2) Componer contexto
    context = "\n\n---\n\n".join(
        [f"[Fuente: {d.metadata.get('source','desconocida')}]\n{(d.page_content or '')[:2000]}" for d in docs]
    )

    # 3) LLM
    messages = [
        SystemMessage(content=SYSTEM_PROMPT),
        HumanMessage(content=f"Pregunta: {query}\n\nContexto:\n{context}")
    ]
    resp = llm.invoke(messages)
    return resp.content, docs

print("✅ ask() actualizado. Usa: answer, docs = ask('tu pregunta')")


✅ ask() actualizado. Usa: answer, docs = ask('tu pregunta')


In [12]:

# Prueba rápida (cambia la pregunta a algo que exista en tus PDFs)
question = "¿Qué indica la introducción del documento principal?"
answer, support_docs = ask(question)
print("🧠 Respuesta:", answer)
print("\n📎 Fuentes:")
for i, d in enumerate(support_docs, 1):
    print(f"{i}.", d.metadata.get("source", "desconocida"))


🧠 Respuesta: La introducción del documento principal indica aspectos relacionados con el despliegue y monitoreo de soluciones de inteligencia artificial generativa, en particular el uso de plantillas de texto para facilitar el comportamiento de los modelos de lenguaje grandes (LLM) dentro de una aplicación, que deben ser desarrolladas, iteradas y gestionadas como parte de una canalización de LLM. También menciona la inclusión de componentes de interfaz de usuario de cara al usuario final como parte del despliegue, y aborda temas como la definición de métricas de evaluación, procesamiento y monitoreo de tablas de inferencia, así como la creación y revisión de monitores y paneles de control para la supervisión de estos sistemas.

📎 Fuentes:
1. C:\Trainings\GenIA_trainings\agentic_rag_openai\docs\generative-ai-deployment-and-monitoring.pdf
2. C:\Trainings\GenIA_trainings\agentic_rag_openai\docs\generative-ai-deployment-and-monitoring.pdf
3. C:\Trainings\GenIA_trainings\agentic_rag_openai\


## ✅ Consejos
- Puedes re‑ejecutar solo las celdas 4→8 para regenerar el vector store tras añadir PDFs.
- Si quieres **reconstruir** Chroma desde cero, elimina la carpeta `chroma_pdfs` antes de ejecutar la celda 6.
- Para usar otro proveedor de embeddings/LLM, reemplaza `OpenAIEmbeddings` / `ChatOpenAI` por el wrapper correspondiente.
- Si tienes PDFs escaneados (imágenes), necesitarás OCR (por ejemplo, `pytesseract` + `pdf2image`) antes de este flujo.
